map

In [12]:
import pandas as pd
import folium
from folium.plugins import MousePosition, Draw
from branca.element import MacroElement
from jinja2 import Template

# Load your GTFS stops.txt file (update path if needed)
stops_df = pd.read_csv("./data/stops.txt")

# Compute map center
center_lat = stops_df['stop_lat'].mean()
center_lon = stops_df['stop_lon'].mean()

# Initialize folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Add stop markers
for _, row in stops_df.iterrows():
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=4,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(
            f"<b>Stop ID:</b> {row['stop_id']}<br>"
            f"<b>Lat:</b> {row['stop_lat']}<br>"
            f"<b>Lon:</b> {row['stop_lon']}",
            max_width=250
        )
    ).add_to(m)

# Show lat/lon on mouse hover
MousePosition(
    position='topright',
    separator=' | ',
    prefix='Lat, Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 6);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 6);}"
).add_to(m)

# Drawing tools (circles for radius, rectangles for bounding boxes)
draw = Draw(
    draw_options={
        "polyline": False,
        "polygon": False,
        "circlemarker": False,
        "marker": True,
        "rectangle": {"shapeOptions": {"color": "#00ff00"}},
        "circle": {"shapeOptions": {"color": "#ff0000"}}
    },
    edit_options={"edit": False}
)
m.add_child(draw)

# JavaScript to add a centroid marker and shape details
class CentroidMarker(MacroElement):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
            var map = {{this._parent.get_name()}};
            map.on('draw:created', function (e) {
                var type = e.layerType,
                    layer = e.layer;

                if (type === 'rectangle') {
                    var bounds = layer.getBounds();
                    var center = bounds.getCenter();
                    
                    // Calculate latitudinal and longitudinal spans
                    var latSpan = bounds.getNorth() - bounds.getSouth();
                    var lonSpan = bounds.getEast() - bounds.getWest();

                    var popupContent = '<b>Rectangle Centroid</b><br>' +
                                     'Lat: ' + center.lat.toFixed(6) + '<br>' +
                                     'Lon: ' + center.lng.toFixed(6) + '<br><br>' +
                                     '<b>Lat Span:</b> ' + latSpan.toFixed(6) + '<br>' +
                                     '<b>Lon Span:</b> ' + lonSpan.toFixed(6);

                    var marker = L.marker([center.lat, center.lng]).addTo(map);
                    marker.bindPopup(popupContent);
                }

                if (type === 'circle') {
                    var center = layer.getLatLng();
                    var radius = layer.getRadius(); // Get the radius in meters

                    var popupContent = '<b>Circle Centroid</b><br>' +
                                     'Lat: ' + center.lat.toFixed(6) + '<br>' +
                                     'Lon: ' + center.lng.toFixed(6) + '<br><br>' +
                                     '<b>Radius:</b> ' + radius.toFixed(2) + ' meters';

                    var marker = L.marker([center.lat, center.lng]).addTo(map);
                    marker.bindPopup(popupContent);
                }
            });
        {% endmacro %}
    """)

    def __init__(self):
        super(CentroidMarker, self).__init__()
        self._name = 'CentroidMarker'

m.add_child(CentroidMarker())


# Save the map as an HTML file
m.save("gtfs_stops_interactive_map_final.html")

print("✅ Map saved as 'gtfs_stops_interactive_map_final.html'. Open it in your browser.")

✅ Map saved as 'gtfs_stops_interactive_map_final.html'. Open it in your browser.


In [16]:
import pandas as pd
import folium
from folium.plugins import MousePosition, Draw
from branca.element import MacroElement, Element
from jinja2 import Template

# Load your GTFS stops.txt file (update path if needed)
stops_df = pd.read_csv("./data/stops.txt")

# Compute map center
center_lat = stops_df['stop_lat'].mean()
center_lon = stops_df['stop_lon'].mean()

# Initialize folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

# Create FeatureGroups
# One for the bus stops, so we can iterate over them
stops_fg = folium.FeatureGroup(name="Bus Stops", show=True)
m.add_child(stops_fg)

# One for the items drawn by the user (shapes and centroid markers)
drawn_fg = folium.FeatureGroup(name="Drawn Shapes", show=True)
m.add_child(drawn_fg)


# Add stop markers to the 'stops_fg' FeatureGroup
for _, row in stops_df.iterrows():
    folium.CircleMarker(
        location=[row['stop_lat'], row['stop_lon']],
        radius=4,
        # Default style for the stops
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=folium.Popup(
            f"<b>Stop ID:</b> {row['stop_id']}<br>"
            f"<b>Lat:</b> {row['stop_lat']}<br>"
            f"<b>Lon:</b> {row['stop_lon']}",
            max_width=250
        )
    ).add_to(stops_fg)

# Show lat/lon on mouse hover
MousePosition(
    position='topright',
    separator=' | ',
    prefix='Lat, Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 6);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 6);}"
).add_to(m)

# Drawing tools
draw = Draw(
    draw_options={
        "polyline": False,
        "polygon": False,
        "circlemarker": False,
        "marker": False, # We create our own markers
        "rectangle": {"shapeOptions": {"color": "#00ff00"}},
        "circle": {"shapeOptions": {"color": "#ff0000"}}
    },
    edit_options={"edit": False}
)
m.add_child(draw)

# Add a LayerControl to toggle FeatureGroups on/off
folium.LayerControl(collapsed=False).add_to(m)


# Custom JavaScript to handle drawing, highlighting, clearing, and navigation
class MapInteraction(MacroElement):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
            
            // Get references to the feature group layers from Python
            var drawnItems = {{this.drawn_items.get_name()}};
            var stopItems = {{this.stop_items.get_name()}};
            var map = {{this._parent.get_name()}};

            // Global variable for the marker created by the 'Go To' function
            var goToMarker = null;

            function highlightStops(drawnLayer) {
                var shapeType = (drawnLayer instanceof L.Circle) ? 'circle' : 'rectangle';
                
                stopItems.eachLayer(function(stop) {
                    var stopLatLng = stop.getLatLng();
                    var isInside = false;

                    if (shapeType === 'circle') {
                        var shapeCenter = drawnLayer.getLatLng();
                        var shapeRadius = drawnLayer.getRadius();
                        if (shapeCenter.distanceTo(stopLatLng) <= shapeRadius) {
                            isInside = true;
                        }
                    } else { // rectangle
                        if (drawnLayer.getBounds().contains(stopLatLng)) {
                            isInside = true;
                        }
                    }

                    if (isInside) {
                        stop.setStyle({color: 'red', fillColor: 'red'});
                    }
                });
            }

            function resetStopStyles() {
                stopItems.eachLayer(function(stop) {
                    stop.setStyle({color: 'blue', fillColor: 'blue'});
                });
            }

            map.on('draw:created', function (e) {
                var type = e.layerType,
                    layer = e.layer;
                
                layer.options.interactive = false;
                drawnItems.addLayer(layer);
                highlightStops(layer);
                
                var center, popupContent, marker;

                if (type === 'rectangle') {
                    var bounds = layer.getBounds();
                    center = bounds.getCenter();
                    var latSpan = bounds.getNorth() - bounds.getSouth();
                    var lonSpan = bounds.getEast() - bounds.getWest();

                    popupContent = '<b>Rectangle Centroid</b><br>' +
                                     'Lat: ' + center.lat.toFixed(6) + '<br>' +
                                     'Lon: ' + center.lng.toFixed(6) + '<br><br>' +
                                     '<b>Lat Span:</b> ' + latSpan.toFixed(6) + '<br>' +
                                     '<b>Lon Span:</b> ' + lonSpan.toFixed(6);
                }

                if (type === 'circle') {
                    center = layer.getLatLng();
                    var radius = layer.getRadius();

                    popupContent = '<b>Circle Centroid</b><br>' +
                                     'Lat: ' + center.lat.toFixed(6) + '<br>' +
                                     'Lon: ' + center.lng.toFixed(6) + '<br><br>' +
                                     '<b>Radius:</b> ' + radius.toFixed(2) + ' meters';
                }
                
                if (center && popupContent) {
                   marker = L.marker([center.lat, center.lng]).bindPopup(popupContent);
                   drawnItems.addLayer(marker);
                }
            });

            // --- Function to clear all drawings and reset stop colors ---
            window.clearDrawings = function() {
                drawnItems.clearLayers();
                resetStopStyles();
                // Also remove the 'Go To' marker if it exists
                if (goToMarker) {
                    map.removeLayer(goToMarker);
                    goToMarker = null;
                }
            }
            
            // --- NEW: Function to go to a specific lat/lon ---
            window.goToLocation = function() {
                var lat = parseFloat(document.getElementById('latInput').value);
                var lon = parseFloat(document.getElementById('lonInput').value);

                if (isNaN(lat) || isNaN(lon) || lat < -90 || lat > 90 || lon < -180 || lon > 180) {
                    alert("Invalid Latitude or Longitude!");
                    return;
                }

                // Remove the previous 'Go To' marker if it exists
                if (goToMarker) {
                    map.removeLayer(goToMarker);
                }
                
                // Fly to the location with a reasonable zoom level
                map.flyTo([lat, lon], 16);

                // Add a new marker at the location
                goToMarker = L.marker([lat, lon]).addTo(map);
                goToMarker.bindPopup('<b>Navigated Location</b><br>Lat: ' + lat + '<br>Lon: ' + lon).openPopup();
            }

        {% endmacro %}
    """)

    # --- THIS METHOD WAS MISSING ---
    def __init__(self, stop_items, drawn_items):
        super(MapInteraction, self).__init__()
        self._name = 'MapInteraction'
        self.stop_items = stop_items
        self.drawn_items = drawn_items
    # ---------------------------------

# --- Control button to clear drawings ---
clear_button_html = """
<div style="position: fixed; top: 10px; left: 150px; z-index:1000;">
  <button onclick="window.clearDrawings()" 
          style="background-color: #f44336; color: white; padding: 10px 15px; 
                 border: none; border-radius: 4px; font-size: 14px; cursor: pointer;">
    Clear Drawings & Highlights
  </button>
</div>
"""
m.get_root().html.add_child(Element(clear_button_html))

# --- NEW: HTML controls for the 'Go To' feature ---
goto_html = """
<div style="position: fixed; top: 10px; left: 360px; z-index:1000; background-color: white; 
            padding: 10px; border: 2px solid grey; border-radius: 5px; font-family: sans-serif;">
  <b style="margin-bottom: 5px; display: block;">Go to Location</b>
  <input type="text" id="latInput" placeholder="Latitude" size="12" 
         style="margin-right: 5px; padding: 5px; border-radius: 3px; border: 1px solid #ccc;"/>
  <input type="text" id="lonInput" placeholder="Longitude" size="12" 
         style="margin-right: 5px; padding: 5px; border-radius: 3px; border: 1px solid #ccc;"/>
  <button onclick="window.goToLocation()" 
          style="background-color: #4CAF50; color: white; padding: 5px 10px; 
                 border: none; border-radius: 4px; cursor: pointer;">Go</button>
</div>
"""
m.get_root().html.add_child(Element(goto_html))


# Add the custom JavaScript interaction logic to the map
m.add_child(MapInteraction(stops_fg, drawn_fg))

# Save the map as an HTML file
m.save("gtfs_stops_interactive_map_full_featured.html")

print("✅ Map saved as 'gtfs_stops_interactive_map_full_featured.html'. Open it in your browser.")

✅ Map saved as 'gtfs_stops_interactive_map_full_featured.html'. Open it in your browser.


In [1]:
import math

# Constants from OneBusAway
RADIUS_OF_EARTH_IN_KM = 6371.01
METERS_PER_DEGREE_AT_EQUATOR = 111319.9
COS_MAX_LAT = math.cos(46 * math.pi / 180)

def distance_accurate(lat1, lon1, lat2, lon2):
    """
    Accurate distance calculation using Haversine formula (great-circle distance)
    Returns distance in meters
    """
    # Convert to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    
    # Calculate delta longitude
    delta_lon = lon2_rad - lon1_rad
    
    # Haversine formula
    y = math.sqrt(
        math.pow(math.cos(lat2_rad) * math.sin(delta_lon), 2) +
        math.pow(math.cos(lat1_rad) * math.sin(lat2_rad) - 
                math.sin(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon), 2)
    )
    x = (math.sin(lat1_rad) * math.sin(lat2_rad) + 
         math.cos(lat1_rad) * math.cos(lat2_rad) * math.cos(delta_lon))
    
    # Calculate distance in meters
    return RADIUS_OF_EARTH_IN_KM * 1000 * math.atan2(y, x)

def distance_faster(lat1, lon1, lat2, lon2):
    """
    Faster but less accurate distance calculation
    Returns distance in meters
    """
    lon_delta = lon2 - lon1
    lat_delta = lat2 - lat1
    return math.sqrt(lon_delta * lon_delta + lat_delta * lat_delta) * METERS_PER_DEGREE_AT_EQUATOR * COS_MAX_LAT


lat1, lon1 = 47.5004692, -122.02224
lat2, lon2 = 47.442950, -121.933884  

# Calculate distances
accurate_dist = distance_accurate(lat1, lon1, lat2, lon2)
faster_dist = distance_faster(lat1, lon1, lat2, lon2)

print(f"Accurate distance: {accurate_dist:.2f} meters")
print(f"Faster distance: {faster_dist:.2f} meters")
print(f"Difference: {abs(accurate_dist - faster_dist):.2f} meters")

Accurate distance: 9220.14 meters
Faster distance: 8152.74 meters
Difference: 1067.40 meters


In [3]:
lat1, lon1 =47.670300419806246,-122.29019165039062
lat2, lon2 = 47.67579094347484, -122.30066299438477

# Calculate distances
accurate_dist = distance_accurate(lat1, lon1, lat2, lon2)
faster_dist = distance_faster(lat1, lon1, lat2, lon2)

print(f"Accurate distance: {accurate_dist:.2f} meters")
print(f"Faster distance: {faster_dist:.2f} meters")
print(f"Difference: {abs(accurate_dist - faster_dist):.2f} meters")

Accurate distance: 993.70 meters
Faster distance: 914.30 meters
Difference: 79.40 meters


In [3]:
import math
def bounds(lat, lon, lat_distance, lon_distance):
    """
    Generate a bounding box (min_lat, min_lon, max_lat, max_lon) around a center point.
    lat, lon: center point in degrees
    lat_distance, lon_distance: distances in meters (radius for latitude and longitude)
    Returns: (min_lat, min_lon, max_lat, max_lon)
    """
    RADIUS_OF_EARTH_IN_KM = 6371.01
    radius_of_earth = RADIUS_OF_EARTH_IN_KM * 1000  # in meters

    lat_radians = math.radians(lat)
    lon_radians = math.radians(lon)

    lat_radius = radius_of_earth
    lon_radius = math.cos(lat_radians) * radius_of_earth

    lat_offset = lat_distance / lat_radius
    lon_offset = lon_distance / lon_radius
    
    print("latSpan = ", 2*lat_offset)
    print("lonSpan = ", 2*lon_offset)

    lat_from = math.degrees(lat_radians - lat_offset)
    lat_to = math.degrees(lat_radians + lat_offset)

    lon_from = math.degrees(lon_radians - lon_offset)
    lon_to = math.degrees(lon_radians + lon_offset)

    return (lat_from, lon_from, lat_to, lon_to)

# Example usage:
center_lat = 47.442950
center_lon = -121.933884
radius_meters = 7000

min_lat, min_lon, max_lat, max_lon = bounds(center_lat, center_lon, radius_meters, radius_meters)
print(f"Bounding box: ({min_lat}, {min_lon}) to ({max_lat}, {max_lon})")

latSpan =  0.002197453778914175
lonSpan =  0.003249114064140654
Bounding box: (47.37999758639658, -122.02696426151594) to (47.50590241360344, -121.84080373848408)


In [1]:
# Stop ID: 64478
# Lat: 47.5004692
# Lon: -122.02224
def isInBound(lat , lon , mintarLan,mintarlong ,maxLat,maxlon):
    return mintarLan<=lat<=maxLat and mintarlong<=lon<=maxlon

In [ ]:
isInBound(47.5004692,-122.02224 ,47.37999758639658, -122.02696426151594,47.50590241360344, -121.84080373848408)

True